In [1]:
import random
import numpy as np
import pandas as pd
import time
import re
import datetime
import os
# import matplotlib.pyplot as plt
# import gensim
# import spacy

In [2]:
import spacy
# pip install -U spacy
# !python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

In [10]:
df = pd.read_excel('Excel_files/Full_Poem_Dataset_9-22_1.xlsx')
print(len(df))
df = df.drop('Unnamed: 0',axis=1)
df.drop_duplicates(subset=['Text'])
print(len(df))
df.head()

records = df.to_dict('records')

2122
2122


## Run Spacy

In [11]:
%%time
badIdxs = []
for i, rec in enumerate(records):
    if i % 100 == 0:
        print(f'{i}/{len(records)} parsed by Spacy.')
    try:
        lines = rec['Text'].split('\n')
        docLines = []
        for line in lines:
            docLines.append(nlp(line))
        rec['docLines'] = docLines
        # rec['doc'] = nlp(rec['Text'])
    except:
        badIdxs.append(i)
for i in badIdxs:
    records.pop(i)

0/2122 parsed by Spacy.
100/2122 parsed by Spacy.
200/2122 parsed by Spacy.
300/2122 parsed by Spacy.
400/2122 parsed by Spacy.
500/2122 parsed by Spacy.
600/2122 parsed by Spacy.
700/2122 parsed by Spacy.
800/2122 parsed by Spacy.
900/2122 parsed by Spacy.
1000/2122 parsed by Spacy.
1100/2122 parsed by Spacy.
1200/2122 parsed by Spacy.
1300/2122 parsed by Spacy.
1400/2122 parsed by Spacy.
1500/2122 parsed by Spacy.
1600/2122 parsed by Spacy.
1700/2122 parsed by Spacy.
1800/2122 parsed by Spacy.
1900/2122 parsed by Spacy.
2000/2122 parsed by Spacy.
2100/2122 parsed by Spacy.
CPU times: user 7min 46s, sys: 27.3 s, total: 8min 13s
Wall time: 10min 47s


In [14]:
# how many tokens?
bTotalTokens = 0
aTotalTokens = 0
for rec in records:
    if rec['Before or after'] == 'Before':
        bTotalTokens += len(rec['docLines'])
    else:
        aTotalTokens += len(rec['docLines'])
bTotalTokens, aTotalTokens

(32838, 37626)

## Text cleaning

In [15]:
def initializeRec(recText):
    if isinstance(recText, str):
        return recText.strip()
    else:
        return False

def skipLine(line, idx):
    if len(line.strip()) == 0:
        return False
    
    # throw hashtag line
    if line.strip()[0] == '#':
        return True
    
    # throw attribution line
    for attr in ['из личного','личный блог','источник:','авторский блог']:
        if attr in line.lower():
            return True
        
    # dots at the beginning
    matches = re.search("[\*\+\^-_][*+^-_= ]+", line.strip()) 
    if matches:
        return True
    
    # is none of it alphanumeric
    containsAlpha = False
    for char in line:
        if char.isalpha():
            containsAlpha = True
            break
    if not containsAlpha:
        return True
    
    return False


def processRec(rec):
    recText = initializeRec(rec['Text'])
    cleanLines = []
    if recText:
        # decide which lines to keep
        lines = recText.split('\n')
        for i, line in enumerate(lines):
            if skipLine(line, i):
                continue
            cleanLines.append(line)
    return '\n'.join(cleanLines)

## Testing out the NLP capabilities

In [29]:
rec = random.choice(records)
text = processRec(rec)
doc = nlp(text)

NERcounter = dict()
for ent in doc.ents:
    lemma = ent.lemma_
    entType = ent.label_
    NERcounter.setdefault(entType, dict())
    NERcounter[entType].setdefault(lemma, 0)
    NERcounter[entType][lemma] += 1
    
print(NERcounter)
print() 
print(text)

{'PER': {'хуёвый': 1}}

молодая женщина в метро
заслоняет от меня своего ребенка
не смотри говорит не смотри
приставляет ладонь словно шору
не смотри на него мой козленочек
вдруг ты сам таким станешь
все люди с цветными волосами пидоры
вам говорят что это неправда
что есть маскулинные геи
истинные защитники охуевшего в жопу отечества
со стальными бицепсами
не курящие марихуану
но послушайте это неправда
это продукт западной пропаганды
призванной сбить вас с пути
истинно говорю вам
все с цветными волосами пидоры
яркие девушки любят пизду
синеволосые мальчики ебутся друг с другом
мы не маргиналы и следим за здоровьем
говорит переодетый в тетку мужик
из локального транс-сообщества
с хуевым как водится макияжем
блондинистым париком тремя высшими
образованиями
но послушайте это неправда
я читаю это обдолбанным
на трезвую голову в жизни
не додумаешься критиковать ваши скрепы
лезть в ваши традиционные семьи
тыкать хуем в гниль и тухлятину
здесь вообще невозможно без трипа
каждый вечер я обесс